In [22]:
import pickle
import pandas as pd
from scipy.sparse import hstack
from scipy.sparse.csr import csr_matrix
import tldextract

In [31]:
with open('chargebee_model_objects.pkl','r') as f:
    tmp = pickle.load(f)
vectorizer,model,bin_boundaries,bin_labels = tmp['vectorizer'],tmp['model'],tmp['bin_boundaries'],tmp['bin_labels']

### dec 28 list for scoring

In [10]:
df1 = pd.read_excel('/home/madan/Desktop/joswin_bck/toPendrive/works/chargebee_enrichment/scoring_lists/dec28/Saas_data_for_scoring_output.xls')
df2 = pd.read_csv('/home/madan/Desktop/joswin_bck/toPendrive/works/chargebee_enrichment/scoring_lists/dec28/Saas_data_for_scoring_output_all_texts.csv',
                 names = ['website','website_text'])

In [13]:
df1['domain'] = df1['website'].apply(lambda x: tldextract.extract(x).domain)
df2['domain'] = df2['website'].apply(lambda x: tldextract.extract(x).domain)

No handlers could be found for logger "tldextract"


In [14]:
df1.head()

,company_linkedin_url,demo_present,emails,id,login_present,match_texts_test,pricing_present,score,urls,website,domain
0,NaN,True,[u'https://conversionista.se/en/conversion-jam...,850,True,"[((u'Pricing',), u""There were some fabulous su...",True,127.5,[u'https://www.twitter.com/jbmoreno/status/251...,http://www.bryaneisenberg.com,bryaneisenberg
1,NaN,True,[],2180,True,"[((u'trial',), u'Reviewsnap Performance Manage...",True,102.4,"[u'https://www.twitter.com/reviewsnap', u'http...",http://www.reviewsnap.com,reviewsnap
2,NaN,True,[],219,True,"[((u'SaaS',), u""We 'cloudify' over 60 best-of-...",False,74.0,[u'https://www.linkedin.com/company/avanan-inc...,http://www.avanan.com,avanan
3,NaN,True,[],218,True,"[((u'SaaS',), u""We 'cloudify' over 60 best-of-...",False,72.5,[u'https://www.linkedin.com/company/avanan-inc...,http://www.avanan.com,avanan
4,NaN,False,[],1557,True,"[((u'subscription',), u'Cancel Anytime\nCancel...",False,60.3,"[u'https://www.facebook.com/lootcrate', u'http...",http://www.lootcrate.com,lootcrate


In [15]:
df2.head()

,website,website_text,domain
0,http://www.101edu.co,101 - Mobile-first learning tools built specif...,101edu
1,http://www.1776.vc,Where Innovation Begins | 1776 Startup Incubat...,1776
2,http://www.1871.com,1871 - Chicago's Technology & Entrepreneurship...,1871
3,http://www.sero.uy,"$ERO, la cuenta para los que no tienen cuenta ...",sero
4,http://www.tapslash.com,NaN,tapslash


In [18]:
df = pd.merge(df1,df2,on='domain')

In [19]:
df.head()

,company_linkedin_url,demo_present,emails,id,login_present,match_texts_test,pricing_present,score,urls,website_x,domain,website_y,website_text
0,NaN,True,[u'https://conversionista.se/en/conversion-jam...,850,True,"[((u'Pricing',), u""There were some fabulous su...",True,127.5,[u'https://www.twitter.com/jbmoreno/status/251...,http://www.bryaneisenberg.com,bryaneisenberg,http://www.bryaneisenberg.com,Bryan & Jeffrey Eisenberg - Professional Speak...
1,NaN,True,[],2180,True,"[((u'trial',), u'Reviewsnap Performance Manage...",True,102.4,"[u'https://www.twitter.com/reviewsnap', u'http...",http://www.reviewsnap.com,reviewsnap,http://www.reviewsnap.com,Employee Performance Management Software Syste...
2,NaN,True,[],219,True,"[((u'SaaS',), u""We 'cloudify' over 60 best-of-...",False,74.0,[u'https://www.linkedin.com/company/avanan-inc...,http://www.avanan.com,avanan,http://www.avanan.com,Avanan - The Cloud Security Platform\n+1.855.5...
3,NaN,True,[],219,True,"[((u'SaaS',), u""We 'cloudify' over 60 best-of-...",False,74.0,[u'https://www.linkedin.com/company/avanan-inc...,http://www.avanan.com,avanan,http://www.avanan.com,Avanan - The Cloud Security Platform\n+1.855.5...
4,NaN,True,[],218,True,"[((u'SaaS',), u""We 'cloudify' over 60 best-of-...",False,72.5,[u'https://www.linkedin.com/company/avanan-inc...,http://www.avanan.com,avanan,http://www.avanan.com,Avanan - The Cloud Security Platform\n+1.855.5...


In [21]:
tfidf = vectorizer.transform(df['website_text'].fillna(''))

In [23]:
tfidf_n_vars = hstack([tfidf,
        csr_matrix(df[['demo_present','login_present','pricing_present']].values.astype(float))])
tfidf_n_vars.shape

(2322, 695)

In [24]:
score = model.predict_proba(tfidf_n_vars)

In [32]:
df['saas_subscription_score'] = score[:,1]
df['saas_subscription_score_bin'] = pd.cut(df['saas_subscription_score'],bin_boundaries,labels=bin_labels)

In [33]:
df_out = df[['id','website_x','domain','login_present','pricing_present','demo_present','score','saas_subscription_score','saas_subscription_score_bin']]
df_out.columns = ['id','website','domain','login_present','pricing_present','demo_present','raw_score','saas_subscription_final_score','saas_subscription_score_bin']

In [34]:
df_out.to_excel('/home/madan/Desktop/joswin_bck/toPendrive/works/chargebee_enrichment/scoring_lists/dec28/Saas_data_for_scoring_with_scores.xls',index=False)

In [35]:
df['saas_subscription_score_bin'].value_counts()

1    357
5    300
4    267
3    253
6    232
7    217
0    192
2    191
9    187
8    126
Name: saas_subscription_score_bin, dtype: int64

In [49]:
#second list
df1 = pd.read_excel('/home/madan/Desktop/joswin_bck/toPendrive/works/chargebee_enrichment/scoring_lists/dec28/Saas_data_for_scoring_1_output.xls')
df2 = pd.read_csv('/home/madan/Desktop/joswin_bck/toPendrive/works/chargebee_enrichment/scoring_lists/dec28/Saas_data_for_scoring_1_output_all_texts.csv',
                 names = ['website','website_text'])

In [50]:
df1['domain'] = df1['website'].apply(lambda x: tldextract.extract(x).domain)
df2['domain'] = df2['website'].apply(lambda x: tldextract.extract(x).domain)

In [51]:
df = pd.merge(df1,df2,on='domain')

In [52]:
tfidf = vectorizer.transform(df['website_text'].fillna(''))
tfidf_n_vars = hstack([tfidf,
        csr_matrix(df[['demo_present','login_present','pricing_present']].values.astype(float))])
tfidf_n_vars.shape

(127, 695)

In [53]:
score = model.predict_proba(tfidf_n_vars)
df['saas_subscription_score'] = score[:,1]
df['saas_subscription_score_bin'] = pd.cut(df['saas_subscription_score'],bin_boundaries,labels=bin_labels)

In [54]:
df_out = df[['id','website_x','domain','login_present','pricing_present','demo_present','score','saas_subscription_score','saas_subscription_score_bin']]
df_out.columns = ['id','website','domain','login_present','pricing_present','demo_present','raw_score','saas_subscription_final_score','saas_subscription_score_bin']

In [55]:
df_out.to_excel('/home/madan/Desktop/joswin_bck/toPendrive/works/chargebee_enrichment/scoring_lists/dec28/Saas_data_for_scoring_1_with_scores.xls',index=False)

In [60]:
pd.DataFrame(df['saas_subscription_score_bin'].value_counts()).sort()

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:1: FutureWarning: sort(....) is deprecated, use sort_index(.....)
  if __name__ == '__main__':


,saas_subscription_score_bin
0,9
1,23
2,6
3,18
4,17
5,16
6,13
7,13
8,0
9,12
